In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Paths and settings
dataset_path = r"C:\Users\saite\OneDrive\Desktop\AUGMENTED"  # Adjust your dataset path
batch_size = 32
image_size = (160, 160)  # Smaller image resolution for faster training
num_epochs = 30
initial_learning_rate = 0.001

from tensorflow.keras.mixed_precision import Policy, set_global_policy

# Set mixed precision policy
policy = Policy('mixed_float16')
set_global_policy(policy)


# Load training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)

# Load validation dataset
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)

# Normalize pixel values
normalization_layer = layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))

# Optimize data pipeline
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)

# Load the base model with pre-trained weights
base_model = MobileNetV2(input_shape=image_size + (3,), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model initially

# Add custom layers
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),  # Regularization to prevent overfitting
    layers.Dense(128, activation='relu'),
    layers.Dense(4, activation='softmax')  # Adjust to match the number of classes
])

# Compile the model with a learning rate schedule
lr_schedule = ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Callbacks: Early stopping and model checkpointing
early_stopping = callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)

checkpoint = callbacks.ModelCheckpoint(
    "best_model.keras",  # Preferred format
    save_best_only=True,
    monitor="val_accuracy",
    mode="max"
)

# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=num_epochs,
    callbacks=[early_stopping, checkpoint]
)

# Fine-tuning: Unfreeze some layers of the base model
base_model.trainable = True
fine_tune_at = len(base_model.layers) // 2  # Unfreeze the top half layers
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Re-compile the model with a lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=initial_learning_rate / 10),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Fine-tune the model
fine_tune_epochs = 10  # Additional epochs for fine-tuning
history_fine = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=fine_tune_epochs,
    initial_epoch=history.epoch[-1],
    callbacks=[early_stopping, checkpoint]
)

# Evaluate the model
loss, accuracy = model.evaluate(val_dataset)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")


Found 10557 files belonging to 4 classes.
Using 8446 files for training.
Found 10557 files belonging to 4 classes.
Using 2111 files for validation.
Epoch 1/30
264/264 ━━━━━━━━━━━━━━━━━━━━ 1658s 6s/step - accuracy: 0.4475 - loss: 1.3665 - val_accuracy: 0.6665 - val_loss: 0.8396
Epoch 2/30
264/264 ━━━━━━━━━━━━━━━━━━━━ 947s 4s/step - accuracy: 0.6354 - loss: 0.9040 - val_accuracy: 0.7039 - val_loss: 0.7600
Epoch 3/30
264/264 ━━━━━━━━━━━━━━━━━━━━ 702s 2s/step - accuracy: 0.6775 - loss: 0.7953 - val_accuracy: 0.7721 - val_loss: 0.6361
Epoch 4/30
264/264 ━━━━━━━━━━━━━━━━━━━━ 484s 2s/step - accuracy: 0.7250 - loss: 0.6988 - val_accuracy: 0.8157 - val_loss: 0.5280
Epoch 5/30
264/264 ━━━━━━━━━━━━━━━━━━━━ 488s 2s/step - accuracy: 0.7588 - loss: 0.6197 - val_accuracy: 0.8290 - val_loss: 0.4973
Epoch 6/30
264/264 ━━━━━━━━━━━━━━━━━━━━ 433s 2s/step - accuracy: 0.7856 - loss: 0.5600 - val_accuracy: 0.8631 - val_loss: 0.4215
Epoch 7/30
264/264 ━━━━━━━━━━━━━━━━━━━━ 517s 2s/step - accuracy: 0.8135 - los

In [4]:
# Save the final trained model
model.save("final_model.keras")  # Save in .keras format
# Or if you prefer the .h5 format
# model.save("final_model.h5")


In [6]:
# Function to preprocess a single image
def preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=image_size)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize
    return img_array

# Path to a new image
image_path = r"C:\Users\saite\OneDrive\Desktop\8bd7b67e02fdfbe0ec875a8815d4bf26.jpg"  

# Preprocess the image
processed_image = preprocess_image(image_path)

# Make a prediction
predictions = model.predict(processed_image)
predicted_class = tf.argmax(predictions[0])  # Get the predicted class index
confidence = tf.reduce_max(predictions[0])  # Get confidence of the prediction

print(f"Predicted Class: {predicted_class}")
print(f"Confidence: {confidence}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted Class: 3
Confidence: 0.50390625
